## Optimal Decision Trees via Integer Programming

In [101]:
import cplex
import numpy as np
import csv
import sys
import random
from copy import deepcopy

### Generating the Tree

In [102]:
class Node:
    def __init__(self,identifier,parent=None,sense=None,splitvar = None,count=0):
        self.__identifier = identifier
        self.__left = None
        self.__right = None

        self.__parent = parent if parent is not None else -1
            
#         if parent is not None:
#             self.__parent = parent
#         else:
#             self.__parent = -1

        if splitvar is not None: self.__splitvar = splitvar
        if sense is not None: self.__sense = sense
        if count is not None: self.__count = count

    @property
    def identifier(self): 
        return self.__identifier

    @property
    def left_child(self):
        return self.__left

    @property
    def right_child(self):
        return self.__right

    @property
    def splitvar(self):
        return self.__splitvar

    @property
    def parent(self):
        return self.__parent

    @property
    def sense(self):
        return self.__sense

    @property
    def count(self):
        return self.__count

    def add_left_child(self,identifier):
        self.__left = identifier

    def add_right_child(self,identifier):
        self.__right = identifier

    def add_splitvar(self,splitvar):
        self.__splitvar = splitvar

    def add_count(self,count):
        self.__count = count
        
        
class Tree:
    def __init__(self):
        self.__nodes = {}

    @property
    def nodes(self):
        return self.__nodes

    def add_node(self, identifier, parent = None, sense = None, splitvar = None):
        node = Node(identifier,parent,sense,splitvar)
        self[identifier] = node

        if parent is not None:
            if sense == 'L':
                self[parent].add_left_child(identifier)
            elif sense == 'R':
                self[parent].add_right_child(identifier)

        return node

    def get_leaf_nodes(self):
        leafs = []
        self._collect_leaf_nodes(self[0],leafs)

        leaves = []
        for leaf in leafs:
            leaves.append(leaf.identifier)
        return leaves

    def _collect_leaf_nodes(self,node,leafs):
        if node is not None:
            if node.left_child == None and node.right_child == None:
                leafs.append(node)
            if node.left_child != None:
                self._collect_leaf_nodes(self[node.left_child],leafs)
            if node.right_child != None:
                self._collect_leaf_nodes(self[node.right_child],leafs)

    def __getitem__(self,key):
        return self.__nodes[key]

    def __setitem__(self,key,item):
        self.__nodes[key] = item
        

In [103]:
def depth_k_tree(k):
    """Generates a balanced decision tree of depth k."""
    tree = Tree()
    tree.add_node(0)
    count = 1
    for j in range(k-1):
        leaves = tree.get_leaf_nodes()
        for leaf in leaves:
            tree.add_node(count,leaf,'L')
            count = count + 1
            tree.add_node(count,leaf,'R')
            count = count + 1

    return tree

tree = depth_k_tree(3)

### Get data from CSV

In [104]:
def get_int_data(csvfile, split = 10.5, gs = False, seed = 0):

    I = A = np.genfromtxt(csvfile, delimiter=',')
    numFeatures, numSamples = len(A[0]) - 1, len(A)    
    group_structure = [] 

    if gs:
        group_structure = I[numSamples-1][:numFeatures]
        I = I[:numSamples-1]

    np.random.seed(seed)
    mask = np.random.choice([False,True],len(I),p=[1-split,split])

    train, test = I[mask], I[~mask]
    train_set, test_set = train[:,:numFeatures], test[:,:numFeatures]
    train_lab, test_lab = train[:,numFeatures], test[:,numFeatures]
    
    return train_set, train_lab, test_set, test_lab, group_structure

In [119]:
trs, trl, tes, tel, gs = get_int_data('./data/monks-1.csv', split=0.7, gs=True)

In [120]:
trs.shape

(290, 17)

### Building the Tree


In [129]:
tree, I, labels, C, maxtime, gs = tree, trs, trl, 1, 1800, gs

In [131]:
# def build_int_model(tree, I, labels, C=1, maxtime=1800, gs = [],filename=' '):
p = cplex.Cplex()
p.objective.set_sense(p.objective.sense.maximize)

numSamples, numFeatures = len(I), len(I[0])

# get group structure from inputs, or else squeeze it out
if gs != []:
    groups = gs
    numGroups = int(max(groups)+1)
else:  # deal with raw binary features
    # clone
    I2 = np.zeros((numSamples,2*numFeatures))
    for j in range(numFeatures):
        I2[:,2*j] = I[:,j]
        I2[:,2*j+1] = 1-I[:,j]
    numGroups = numFeatures
    I = I2
    groups = np.zeros(2*numFeatures)
    for j in range(numGroups):
        groups[2*j] = groups[2*j+1] = j
#             groups[2*j+1] = j
    numFeatures = 2*numFeatures

leaves = tree.get_leaf_nodes() # are actually decison nodes(index) adjacent to leaves
numNodes = max(leaves) + 1 # Since start nodes labeled begining with 0
print ('number of nodes is',numNodes)
print ('leaves are',leaves)

# set up the buckets
numBuckets = 0
bucket_dict = {}
for leaf in leaves:
    bucket_dict[leaf] = numBuckets
    numBuckets += 1
print ('buckets is',bucket_dict)

# establish the variables
names = []
for k in range(numNodes):
    for j in range(numFeatures):
        name = 'z_'+repr(j)+'_'+repr(k)
        names.append(name)
z_ = numNodes*numFeatures

for k in range(numNodes):
    for g in range(numGroups):
        name = 'v_'+repr(g)+'_'+repr(k)
        names.append(name)
zv_ = z_ + numNodes*numGroups

for b in range(2*numBuckets):
    for i in range(numSamples):
        name = 'c_'+repr(i)+'_'+repr(b)
        names.append(name)
zvc_ = zv_ + 2*numBuckets*numSamples

# path enumeration
paths = []
senses = []
for leaf in leaves:
    curr = leaf
    path = []
    path.append(curr)
    leafsense = []
    while curr != 0:
        leafsense.append(tree[curr].sense)
        curr = tree[curr].parent
        path.append(curr)
    paths.append(path)
    senses.append(leafsense)
print('paths are',paths)
print('senses are',senses)
A = np.zeros((numNodes*((numNodes+1)*numSamples + numGroups*numFeatures+2)+1,zvc_))
rhs = []
ineq = ''
constraint_cnt = 0
cnames = []

# pick a group at each node   \sum v_g^k=1
for k in range(numNodes):
    for g in range(numGroups):
        A[constraint_cnt,z_ + k*numGroups + g] = 1
    rhs.append(1.0)

    ineq = ineq + "E"
    constraint_cnt = constraint_cnt + 1

# group hierarchy constraints:
group_no = 0
curr_feature = 0
print('groups is',groups)
for j in range(numFeatures):
    ind = int(groups[j])
    if ind != group_no or j == (numFeatures - 1):
        prev_feature = curr_feature
        if j == (numFeatures - 1):
            curr_feature = numFeatures
        else:
            curr_feature = j
        for k in range(numNodes):
            for j in range(prev_feature,curr_feature):
                A[constraint_cnt,z_ + k*numGroups + group_no] = -1
                A[constraint_cnt,k*numFeatures + j] = 1

                rhs.append(0.0)

                ineq = ineq + "L"
                constraint_cnt += 1
        group_no = ind


# left tree constraints
for k in range(numNodes):
    collected_buckets = []
    sensescopy = deepcopy(senses)

    for sense in sensescopy:
        sense.insert(0,'L')

    for path in range(len(paths)):
        if k in paths[path]:
            ind = paths[path].index(k)
            if sensescopy[path][ind] == 'L':
                collected_buckets.append(bucket_dict[paths[path][0]])
            if sensescopy[path][ind] == 'L' and ind != 0:
                collected_buckets.append(numBuckets + bucket_dict[paths[path][0]])

    for i in range(numSamples):
        for c in collected_buckets:
            for j in range(numFeatures):
                A[constraint_cnt,k*numFeatures + j] = -I[i][j]
            A[constraint_cnt,zv_ + c*numSamples + i] = 1
            rhs.append(0.0)
            ineq = ineq + "L"
            constraint_cnt = constraint_cnt + 1

# right tree constraints
for k in range(numNodes):
    collected_buckets = []
    sensescopy = deepcopy(senses)
    for sense in sensescopy:
        sense.insert(0,'R')
    for path in range(len(paths)):
        if k in paths[path]:
            ind = paths[path].index(k)
            if sensescopy[path][ind] == 'R':
                collected_buckets.append(numBuckets + bucket_dict[paths[path][0]])
            if sensescopy[path][ind] == 'R' and ind != 0:
                collected_buckets.append(bucket_dict[paths[path][0]])
    for i in range(numSamples):
        for c in collected_buckets:
            for j in range(numFeatures):
                A[constraint_cnt,k*numFeatures + j] = I[i][j]
            A[constraint_cnt,zv_ + c*numSamples + i] = 1
            rhs.append(1.0)
            ineq = ineq + "L"
            constraint_cnt = constraint_cnt + 1
#                 cname = 'BucketConstraintRight_'+repr(k)+'_Sample_'+repr(i)+'_Bucket_'+repr(c)
#                 cnames.append(cname)

print("Number of rows: %s " % constraint_cnt)
numRows = constraint_cnt
numCols = zvc_
print("Number of columns: %s " % numCols)

indices = [[i for i in range(numRows) if A[i,j] != 0] for j in range(numCols)]
values = [[A[i,j] for i in range(numRows) if A[i,j] != 0] for j in range(numCols)]
cols = [[indices[i],values[i]] for i in range(numCols)]

rhs = np.array(rhs)
senses = ineq

# define the objective
obj = np.zeros(numCols)

for i in range(numSamples):
    if labels[i] == 0:
        for b in range(numBuckets):
            obj[zv_ + b*numSamples + i] = C
    else:
        for b in range(numBuckets):
            obj[zv_ + numBuckets*numSamples + b*numSamples + i] = 1

# set up types, priorities
priority_vec = []


types = numNodes*(numGroups+numFeatures)*'I'
for j in range(numNodes*numFeatures):
    priority_vec.append((j,1,p.order.branch_direction.down))


types = types + 2*numBuckets*numSamples*'I'


lb, ub = np.zeros(numCols), np.ones(numCols)

# Load into p
p.linear_constraints.add(rhs=rhs,senses=senses)
p.variables.add(obj=obj,lb=lb,ub=ub,columns=cols,types=types)
p.parameters.timelimit.set(maxtime)
p.solve()
sol = p.solution
trial = sol.get_objective_value()

# what are the groups in the solution?
solgroups = []
splits = [[] for k in range(numNodes)]

for k in range(numNodes):
    for j in range(z_ + k*numGroups,z_ + (k+1)*numGroups):
        if sol.get_values(j) > 0.99:
            group = j - z_ - k*numGroups
            solgroups.append(group)
for k in range(numNodes):
    indices = [i for i,x in enumerate(groups) if x == solgroups[k]]
    for ind in indices:
        dir = sol.get_values(k*numFeatures + ind)
        if dir > 0.99:
            splits[k].append(ind)
    tree[k].add_splitvar(splits[k])


/Users/jacob/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  


number of nodes is 7
leaves are [3, 4, 5, 6]
buckets is {3: 0, 4: 1, 5: 2, 6: 3}
paths are [[3, 1, 0], [4, 1, 0], [5, 2, 0], [6, 2, 0]]
senses are [['L', 'L'], ['R', 'L'], ['L', 'R'], ['R', 'R']]
groups is [0. 0. 0. 1. 1. 1. 2. 2. 3. 3. 3. 4. 4. 4. 4. 5. 5.]
Number of rows: 7086 
Number of columns: 2481 
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               1800
Found incumbent of value 0.000000 after 0.00 sec. (0.67 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 2908 rows and 1164 columns.
MIP Presolve modified 870 coefficients.
Aggregator did 7 substitutions.
Reduced MIP has 4171 rows, 1310 columns, and 27790 nonzeros.
Reduced MIP has 1310 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.15 sec. (133.67 ticks)
Probing time = 0.01 sec. (3.93 ticks)
Tried aggregator 1 time.
Reduced MIP has 4171 rows, 1310 columns, and 27790 nonzeros.
Reduced MIP has 1310 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve tim

In [135]:
len(p.solution.get_values())

2481

### Run the model (this can take a very long time)

In [125]:
build_int_model(tree, trs, trl, gs=gs)

/Users/jacob/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  


number of nodes is 7
leaves are [3, 4, 5, 6]
buckets is {3: 0, 4: 1, 5: 2, 6: 3}
paths are [[3, 1, 0], [4, 1, 0], [5, 2, 0], [6, 2, 0]]
senses are [['L', 'L'], ['R', 'L'], ['L', 'R'], ['R', 'R']]
groups is [0. 0. 0. 1. 1. 1. 2. 2. 3. 3. 3. 4. 4. 4. 4. 5. 5.]
Number of rows: 7086 
Number of columns: 2481 
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               1800
Found incumbent of value 0.000000 after 0.00 sec. (0.67 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 2908 rows and 1164 columns.
MIP Presolve modified 870 coefficients.
Aggregator did 7 substitutions.
Reduced MIP has 4171 rows, 1310 columns, and 27790 nonzeros.
Reduced MIP has 1310 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.14 sec. (133.67 ticks)
Probing time = 0.01 sec. (3.93 ticks)
Tried aggregator 1 time.
Reduced MIP has 4171 rows, 1310 columns, and 27790 nonzeros.
Reduced MIP has 1310 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve tim

NameError: name 'numcols' is not defined

In [117]:
def test_int_model(tree, I, labels, gs=False):
    
    numSamples, numFeatures = len(I), len(I[0])

    if not gs:
        # clone
        I2 = np.zeros((numSamples,2*numFeatures))
        for j in range(numFeatures):
            I2[:,2*j] = I[:,j]
            I2[:,2*j+1] = 1-I[:,j]
        numGroups = numFeatures
        I = I2

    numSamples = len(I)
    type1 = type2 = 0
    numGood = np.count_nonzero(labels)
    numBad = numSamples - numGood
    correct_inds = [False]*numSamples

    for sample in range(numSamples):
        k, prevk, fullstop, feats = 0, 0, 0, I[sample]
        while fullstop == 0:
            stop = 0
            for lefty in tree[k].splitvar:
                if I[sample][lefty] == 1 and stop == 0:
                    prevk = k
                    k = tree[k].left_child
                    stop = 1
            if stop == 0:
                prevk = k
                k = tree[k].right_child
            if k is None:
                fullstop = 1
        stop = 0
        for lefty in tree[prevk].splitvar:
            if I[sample][lefty] == 1 and stop == 0:
                stop = 1
                if labels[sample] == 0:
                    type2 = type2 + 1
                    correct_inds[sample] = True
        if stop == 0 and labels[sample] == 1:
            type1 = type1 + 1
            correct_inds[sample]= True
    recall = float(type1)/numGood
    spec = float(type2)/numBad
    acc = float(type1+type2)/numSamples

    return {'recall':recall, 'specificity': spec, 'accuracy': acc, 'inds':correct_inds}



In [122]:
res = test_int_model(tree, tes, tel, gs=True)
print('test accuracy is', res['accuracy'])

test accuracy is 0.8309859154929577


In [84]:
tree.get_leaf_nodes()

[3, 4, 5, 6]